## 1. Importação de Bibliotecas

In [ ]:
# Bibliotecas principais
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

# Scikit-learn
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import roc_curve, auc

# NLTK
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Datasets
from datasets import load_dataset

# Configurações
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✓ Bibliotecas importadas com sucesso!")

## 2. Download de Recursos NLTK

In [ ]:
# Download de recursos necessários do NLTK
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('omw-1.4', quiet=True)

print("✓ Recursos NLTK baixados com sucesso!")

## 3. Carregamento do Dataset

In [ ]:
# Carregar o dataset IMDb
print("Carregando dataset IMDb...")
dataset = load_dataset('imdb')

# Converter para DataFrame
train_df = pd.DataFrame(dataset['train'])
test_df = pd.DataFrame(dataset['test'])

print(f"\n✓ Dataset carregado com sucesso!")
print(f"  - Tamanho do conjunto de treino: {len(train_df)}")
print(f"  - Tamanho do conjunto de teste: {len(test_df)}")
print(f"\nPrimeiras linhas do dataset:")
train_df.head()

## 4. Análise Exploratória dos Dados

In [ ]:
# Informações básicas
print("Informações do Dataset:")
print(train_df.info())
print("\nEstatísticas Descritivas:")
print(train_df.describe())

In [ ]:
# Distribuição das classes
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Gráfico de barras
train_df['label'].value_counts().plot(kind='bar', ax=axes[0], color=['#ff6b6b', '#4ecdc4'])
axes[0].set_title('Distribuição das Classes - Treino', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Sentimento (0=Negativo, 1=Positivo)', fontsize=12)
axes[0].set_ylabel('Quantidade', fontsize=12)
axes[0].set_xticklabels(['Negativo', 'Positivo'], rotation=0)

# Gráfico de pizza
train_df['label'].value_counts().plot(kind='pie', ax=axes[1], autopct='%1.1f%%', 
                                       colors=['#ff6b6b', '#4ecdc4'], startangle=90)
axes[1].set_title('Proporção das Classes', fontsize=14, fontweight='bold')
axes[1].set_ylabel('')
axes[1].legend(['Negativo', 'Positivo'])

plt.tight_layout()
plt.show()

print(f"\nBalanceamento das classes:")
print(train_df['label'].value_counts())

In [ ]:
# Análise do tamanho dos textos
train_df['text_length'] = train_df['text'].apply(len)
train_df['word_count'] = train_df['text'].apply(lambda x: len(x.split()))

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Distribuição do tamanho dos textos
axes[0].hist(train_df['text_length'], bins=50, color='#4ecdc4', alpha=0.7, edgecolor='black')
axes[0].set_title('Distribuição do Tamanho dos Textos (caracteres)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Número de Caracteres', fontsize=12)
axes[0].set_ylabel('Frequência', fontsize=12)
axes[0].axvline(train_df['text_length'].mean(), color='red', linestyle='--', label=f'Média: {train_df["text_length"].mean():.0f}')
axes[0].legend()

# Distribuição do número de palavras
axes[1].hist(train_df['word_count'], bins=50, color='#ff6b6b', alpha=0.7, edgecolor='black')
axes[1].set_title('Distribuição do Número de Palavras', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Número de Palavras', fontsize=12)
axes[1].set_ylabel('Frequência', fontsize=12)
axes[1].axvline(train_df['word_count'].mean(), color='red', linestyle='--', label=f'Média: {train_df["word_count"].mean():.0f}')
axes[1].legend()

plt.tight_layout()
plt.show()

print(f"\nEstatísticas do tamanho dos textos:")
print(f"  - Média de caracteres: {train_df['text_length'].mean():.2f}")
print(f"  - Média de palavras: {train_df['word_count'].mean():.2f}")
print(f"  - Máximo de caracteres: {train_df['text_length'].max()}")
print(f"  - Máximo de palavras: {train_df['word_count'].max()}")

## 5. Pré-processamento de Texto

In [ ]:
import re
from html import unescape

def preprocess_text(text):
    """
    Pré-processa o texto:
    1. Remove HTML tags
    2. Converte para minúsculas
    3. Remove caracteres especiais
    4. Remove stopwords
    5. Aplica lematização
    """
    # Decodificar entidades HTML
    text = unescape(text)
    
    # Remover HTML tags
    text = re.sub(r'<.*?>', '', text)
    
    # Converter para minúsculas
    text = text.lower()
    
    # Remover URLs
    text = re.sub(r'http\S+|www\S+', '', text)
    
    # Remover caracteres especiais e números (mantém apenas letras)
    text = re.sub(r'[^a-z\s]', '', text)
    
    # Tokenização
    tokens = word_tokenize(text)
    
    # Remover stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words and len(word) > 2]
    
    # Lematização
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    return ' '.join(tokens)

# Teste da função
sample_text = train_df['text'].iloc[0][:200]
print("Texto original:")
print(sample_text)
print("\nTexto processado:")
print(preprocess_text(sample_text))

In [ ]:
# Aplicar pré-processamento (usando uma amostra para demonstração)
# Para o dataset completo, isso pode levar alguns minutos

# Usar uma amostra menor para demonstração (remova o .sample() para usar o dataset completo)
SAMPLE_SIZE = 5000  # Use None para processar todo o dataset

if SAMPLE_SIZE:
    train_sample = train_df.sample(n=SAMPLE_SIZE, random_state=42)
    test_sample = test_df.sample(n=SAMPLE_SIZE, random_state=42)
    print(f"\nUsando amostra de {SAMPLE_SIZE} exemplos para treino e teste")
else:
    train_sample = train_df
    test_sample = test_df
    print(f"\nUsando dataset completo")

print("\nProcessando textos de treino...")
train_sample['processed_text'] = train_sample['text'].progress_apply(preprocess_text)

print("Processando textos de teste...")
test_sample['processed_text'] = test_sample['text'].progress_apply(preprocess_text)

print("\n✓ Pré-processamento concluído!")

## 6. Vetorização com TF-IDF

In [ ]:
# Preparar dados
X_train = train_sample['processed_text']
y_train = train_sample['label']
X_test = test_sample['processed_text']
y_test = test_sample['label']

# Criar vetorizador TF-IDF
print("Criando vetores TF-IDF...")
tfidf_vectorizer = TfidfVectorizer(
    max_features=5000,  # Limitar a 5000 features mais importantes
    min_df=2,           # Palavra deve aparecer em pelo menos 2 documentos
    max_df=0.8,         # Palavra não deve aparecer em mais de 80% dos documentos
    ngram_range=(1, 2)  # Usar unigramas e bigramas
)

# Transformar textos em vetores
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

print(f"\n✓ Vetorização concluída!")
print(f"  - Shape do conjunto de treino: {X_train_tfidf.shape}")
print(f"  - Shape do conjunto de teste: {X_test_tfidf.shape}")
print(f"  - Número de features: {len(tfidf_vectorizer.get_feature_names_out())}")

In [ ]:
# Visualizar as palavras mais importantes
feature_names = tfidf_vectorizer.get_feature_names_out()
print("\nExemplos de features (palavras e bigramas):")
print(feature_names[:20])
print("...")
print(feature_names[-20:])

## 7. Treinamento dos Modelos

### 7.1 Regressão Logística

In [ ]:
print("Treinando Regressão Logística...")
lr_model = LogisticRegression(max_iter=1000, random_state=42, n_jobs=-1)
lr_model.fit(X_train_tfidf, y_train)

# Fazer predições
y_pred_lr = lr_model.predict(X_test_tfidf)

# Avaliar modelo
lr_accuracy = accuracy_score(y_test, y_pred_lr)
print(f"\n✓ Regressão Logística treinada!")
print(f"  Acurácia: {lr_accuracy:.4f} ({lr_accuracy*100:.2f}%)")

### 7.2 Naive Bayes

In [ ]:
print("Treinando Naive Bayes...")
nb_model = MultinomialNB()
nb_model.fit(X_train_tfidf, y_train)

# Fazer predições
y_pred_nb = nb_model.predict(X_test_tfidf)

# Avaliar modelo
nb_accuracy = accuracy_score(y_test, y_pred_nb)
print(f"\n✓ Naive Bayes treinado!")
print(f"  Acurácia: {nb_accuracy:.4f} ({nb_accuracy*100:.2f}%)")

## 8. Avaliação dos Modelos

In [ ]:
# Comparação de acurácia
models_comparison = pd.DataFrame({
    'Modelo': ['Regressão Logística', 'Naive Bayes'],
    'Acurácia': [lr_accuracy, nb_accuracy]
})

print("\n" + "="*50)
print("COMPARAÇÃO DOS MODELOS")
print("="*50)
print(models_comparison.to_string(index=False))
print("="*50)

# Visualizar comparação
plt.figure(figsize=(10, 6))
bars = plt.bar(models_comparison['Modelo'], models_comparison['Acurácia'], 
               color=['#4ecdc4', '#ff6b6b'], alpha=0.8, edgecolor='black', linewidth=1.5)
plt.title('Comparação de Acurácia dos Modelos', fontsize=16, fontweight='bold')
plt.ylabel('Acurácia', fontsize=12)
plt.ylim(0.5, 1.0)
plt.grid(axis='y', alpha=0.3)

# Adicionar valores nas barras
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
             f'{height:.4f}\n({height*100:.2f}%)',
             ha='center', va='bottom', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
# Relatório de classificação para Regressão Logística
print("\n" + "="*50)
print("REGRESSÃO LOGÍSTICA - Relatório Detalhado")
print("="*50)
print(classification_report(y_test, y_pred_lr, target_names=['Negativo', 'Positivo']))

In [ ]:
# Relatório de classificação para Naive Bayes
print("\n" + "="*50)
print("NAIVE BAYES - Relatório Detalhado")
print("="*50)
print(classification_report(y_test, y_pred_nb, target_names=['Negativo', 'Positivo']))

In [ ]:
# Matrizes de confusão
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Matriz de confusão - Regressão Logística
cm_lr = confusion_matrix(y_test, y_pred_lr)
sns.heatmap(cm_lr, annot=True, fmt='d', cmap='Blues', ax=axes[0], 
            xticklabels=['Negativo', 'Positivo'],
            yticklabels=['Negativo', 'Positivo'])
axes[0].set_title('Matriz de Confusão - Regressão Logística', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Valor Real', fontsize=12)
axes[0].set_xlabel('Valor Predito', fontsize=12)

# Matriz de confusão - Naive Bayes
cm_nb = confusion_matrix(y_test, y_pred_nb)
sns.heatmap(cm_nb, annot=True, fmt='d', cmap='Reds', ax=axes[1],
            xticklabels=['Negativo', 'Positivo'],
            yticklabels=['Negativo', 'Positivo'])
axes[1].set_title('Matriz de Confusão - Naive Bayes', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Valor Real', fontsize=12)
axes[1].set_xlabel('Valor Predito', fontsize=12)

plt.tight_layout()
plt.show()

## 9. Análise de Features Importantes

In [ ]:
# Palavras mais importantes para cada classe (Regressão Logística)
feature_names = np.array(tfidf_vectorizer.get_feature_names_out())
coef = lr_model.coef_[0]

# Top palavras positivas
top_positive_indices = np.argsort(coef)[-20:]
top_positive_words = feature_names[top_positive_indices]
top_positive_scores = coef[top_positive_indices]

# Top palavras negativas
top_negative_indices = np.argsort(coef)[:20]
top_negative_words = feature_names[top_negative_indices]
top_negative_scores = coef[top_negative_indices]

# Visualização
fig, axes = plt.subplots(1, 2, figsize=(16, 8))

# Palavras positivas
axes[0].barh(range(len(top_positive_words)), top_positive_scores, color='#4ecdc4', alpha=0.8)
axes[0].set_yticks(range(len(top_positive_words)))
axes[0].set_yticklabels(top_positive_words)
axes[0].set_xlabel('Peso (Coeficiente)', fontsize=12)
axes[0].set_title('Top 20 Palavras Associadas a Sentimento POSITIVO', fontsize=14, fontweight='bold')
axes[0].invert_yaxis()

# Palavras negativas
axes[1].barh(range(len(top_negative_words)), top_negative_scores, color='#ff6b6b', alpha=0.8)
axes[1].set_yticks(range(len(top_negative_words)))
axes[1].set_yticklabels(top_negative_words)
axes[1].set_xlabel('Peso (Coeficiente)', fontsize=12)
axes[1].set_title('Top 20 Palavras Associadas a Sentimento NEGATIVO', fontsize=14, fontweight='bold')
axes[1].invert_yaxis()

plt.tight_layout()
plt.show()

print("\nTop 10 palavras POSITIVAS:")
for word, score in zip(top_positive_words[-10:], top_positive_scores[-10:]):
    print(f"  {word:20s}: {score:.4f}")

print("\nTop 10 palavras NEGATIVAS:")
for word, score in zip(top_negative_words[:10], top_negative_scores[:10]):
    print(f"  {word:20s}: {score:.4f}")

## 10. Teste com Novos Textos

In [ ]:
def predict_sentiment(text, model=lr_model):
    """
    Prediz o sentimento de um texto
    """
    # Pré-processar texto
    processed = preprocess_text(text)
    
    # Vetorizar
    vectorized = tfidf_vectorizer.transform([processed])
    
    # Predizer
    prediction = model.predict(vectorized)[0]
    probability = model.predict_proba(vectorized)[0]
    
    sentiment = "POSITIVO" if prediction == 1 else "NEGATIVO"
    confidence = probability[prediction] * 100
    
    return sentiment, confidence

# Exemplos de teste
test_reviews = [
    "This movie was absolutely amazing! The acting was superb and the plot was engaging.",
    "Terrible film. Waste of time and money. I couldn't even finish watching it.",
    "It was okay, nothing special but not terrible either.",
    "Best movie I've seen in years! Highly recommended!",
    "Boring and predictable. The worst movie of the year."
]

print("="*80)
print("TESTE COM NOVOS TEXTOS")
print("="*80)

for i, review in enumerate(test_reviews, 1):
    sentiment, confidence = predict_sentiment(review)
    print(f"\nReview {i}: {review}")
    print(f"Sentimento: {sentiment} (Confiança: {confidence:.2f}%)")
    print("-" * 80)

## 11. Salvando o Modelo

In [ ]:
import pickle
import os

# Criar diretório se não existir
os.makedirs('../models', exist_ok=True)

# Salvar modelo
with open('../models/sentiment_model.pkl', 'wb') as f:
    pickle.dump(lr_model, f)

# Salvar vetorizador
with open('../models/tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(tfidf_vectorizer, f)

print("✓ Modelo e vetorizador salvos com sucesso!")
print("  - Modelo: models/sentiment_model.pkl")
print("  - Vetorizador: models/tfidf_vectorizer.pkl")

## 12. Conclusões

### Resultados Obtidos:

1. **Regressão Logística**: Apresentou excelente performance na classificação de sentimentos
2. **Naive Bayes**: Modelo mais rápido, com boa acurácia

### Principais Descobertas:

- O dataset IMDb é bem balanceado, facilitando o treinamento
- Palavras como "excellent", "great", "best" são fortes indicadores de sentimento positivo
- Palavras como "worst", "bad", "terrible" são fortes indicadores de sentimento negativo
- A vetorização TF-IDF capturou bem as características importantes do texto

### Possíveis Melhorias:

1. Usar modelos mais avançados (LSTM, BERT, Transformers)
2. Aumentar o tamanho do dataset de treino
3. Aplicar técnicas de data augmentation
4. Fazer fine-tuning de hiperparâmetros
5. Considerar análise de aspectos específicos (acting, plot, cinematography)

### Aplicações Práticas:

- Sistema de recomendação de filmes
- Análise de feedback de clientes
- Monitoramento de reputação online
- Dashboard de análise de sentimentos em tempo real